<a href="https://colab.research.google.com/github/Cicciokr/latin-ai-model/blob/main/Fine_Tuning_Transformer_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
#!pip install evaluate
#!pip install rouge_score
!pip install wandb


import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForMaskedLM, TrainerCallback, EarlyStoppingCallback, get_scheduler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
#from numba import cuda
import math
import torch
import wandb
import os
os.environ["WANDB_PROJECT"] = "TrainingRoberta"
wandb.login(key="e115f18967efc277b293c042cb216e776d82a741")
wandb.init()
#import evaluate
#rouge = evaluate.load('rouge')

#dataset testo
#dataset = load_dataset('text', data_files='la.txt')
#dataset parquet
#dataset = load_dataset("Cicciokr/CC-100-Latin", revision="refs/convert/parquet")
MODEL_NAME = "ClassCat/roberta-base-latin-v2"

#device_cuda = cuda.get_current_device()
#device_cuda.reset()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForMaskedLM.from_pretrained(MODEL_NAME)
model.to(device)

def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

#dataset = load_dataset("parquet", data_dir="./parquet", trust_remote_code=True)
#tokenized_dataset = dataset.map(preprocess_function, batched=True, num_proc=4)
#tokenized_dataset.save_to_disk("./dataset/tokenized_dataset")
#tokenized_dataset = load_from_disk("./dataset_light/tokenized_dataset")
dataset = load_dataset("pstroe/cc100-latin", data_files="la.nolorem.tok.latalphabetonly.v2.json", field="train")
print(dataset)
#Eseguo lo split tra train e test applicando lo shuffle
dataset_split = dataset['train'].train_test_split(test_size=0.003, train_size=0.01, shuffle=True)
#dataset_split_train = dataset['train'].train_test_split(test_size=0.0001, shuffle=True)
# Applicare la tokenizzazione
tokenized_datasets_test = dataset_split['test'].map(preprocess_function, batched=True, num_proc=4)
tokenized_datasets_train = dataset_split['train'].map(preprocess_function, batched=True, num_proc=4)
print(tokenized_datasets_train)
print(tokenized_datasets_test)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,                   # Abilita il mascheramento
    mlm_probability=0.15        # Percentuale di token da mascherare
)

#il 20% dei dati viene usato come test e l'80% viene usato come train, per evitare overfitting
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Colab Notebooks/TrainingLog",
    run_name="latinroberta",
    save_strategy="steps",
    eval_strategy="steps",
    save_steps=10,
    eval_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    num_train_epochs=3,
    weight_decay=0.01,
    max_grad_norm=1.0,
    logging_dir="/content/drive/MyDrive/Colab Notebooks/TrainingLog/logs",
    fp16=True,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=24,
    logging_steps=100,
    warmup_steps=1000,
    save_total_limit=3,
    greater_is_better=False,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    overwrite_output_dir=True,
    report_to="wandb",
    optim="adamw_torch",
    lr_scheduler_type="linear"
)

#metric = evaluate.load("accuracy")
def remove_values_from_list(the_list, val):
   return [value for value in the_list if value != val]


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets_train,
    eval_dataset=tokenized_datasets_test,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    tokenizer=tokenizer
)

trainer.train()

model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/latinmlmroberta")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/latinmlmroberta")

#results_eval = trainer.evaluate()
#print(results_eval)

wandb.finish()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: cicciokr (cicciokr-universit-degli-studi-guglielmo-marconi). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/431 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/845k [00:00<?, ?B/s]

...

Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 934
})
Dataset({
    features: ['text', 'input_ids', 'attention_mask'],
    num_rows: 187
})


<ipython-input-1-8b0deba3e9ca>:99: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss,Validation Loss
10,No log,3.974721
20,No log,3.713460


There were missing keys in the checkpoint model loaded: ['lm_head.decoder.weight', 'lm_head.decoder.bias'].


eval/loss,█▁
eval/runtime,▁█
eval/samples_per_second,█▁
eval/steps_per_second,█▁
train/epoch,▁▅█
train/global_step,▁▅█
eval/loss,3.71346
eval/runtime,1.3726
eval/samples_per_second,136.235
eval/steps_per_second,5.828
total_flos,707655377092608.0
